### Importe del dataset



---



Dataset sacado de kaggle ["Uber Data Analytics Dashboard"](https://www.kaggle.com/datasets/yashdevladdha/uber-ride-analytics-dashboard)

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yashdevladdha/uber-ride-analytics-dashboard")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/uber-ride-analytics-dashboard


In [2]:
import os
import pandas as pd

os.listdir(path)
df = pd.read_csv(os.path.join(path, "ncr_ride_bookings.csv"))

### Visualización de los primeros 5 registros



---



In [3]:
df.head(5)

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


Se puede ver que hay varios registros con valores nulos, entonces nos podemos concentrar en registros que no contengan valores nulos. Para eso veamos cuantos valores nulos hay en cada columan.

### Limpieza de datos



---



In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               150000 non-null  object 
 1   Time                               150000 non-null  object 
 2   Booking ID                         150000 non-null  object 
 3   Booking Status                     150000 non-null  object 
 4   Customer ID                        150000 non-null  object 
 5   Vehicle Type                       150000 non-null  object 
 6   Pickup Location                    150000 non-null  object 
 7   Drop Location                      150000 non-null  object 
 8   Avg VTAT                           139500 non-null  float64
 9   Avg CTAT                           102000 non-null  float64
 10  Cancelled Rides by Customer        10500 non-null   float64
 11  Reason for cancelling by Customer  1050

Antes de empezar, primero vemos si hay valores duplicados.

In [29]:
print(df.duplicated().sum())

0


No hay valores duplicados! Por lo que seguimos con la limpieza.

#### Limpieza de filas



---



Al ver los nombres de todas las columnas, si tomamos como referencia 'Booking Status', podemos ver lo siguiente:

In [20]:
df['Booking Status'].value_counts()

,count
Booking Status,
Completed,93000


Podemos notar que el Status de 'Completed' son los que tienen más viajes, entonces para este análisis se usarán únicamente los registros de 'Completed'. Para ello eliminaremos los demás status

In [21]:
df.drop(df[df['Booking Status'] == 'Cancelled by Driver'].index, inplace=True)

In [22]:
df.drop(df[df['Booking Status'] == 'No Driver Found'].index, inplace=True)

In [23]:
df.drop(df[df['Booking Status'] == 'Cancelled by Customer'].index, inplace=True)

In [24]:
df.drop(df[df['Booking Status'] == 'Incomplete'].index, inplace=True)

Volvemos a ver los primeros registros

In [25]:
df.head()

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,737.0,48.21,4.1,4.3,UPI
5,2024-02-06,09:44:56,"""CNR4096693""",Completed,"""CID4670564""",Auto,AIIMS,Narsinghpur,5.1,18.1,316.0,4.85,4.1,4.6,UPI
6,2024-06-17,15:45:58,"""CNR2002539""",Completed,"""CID6800553""",Go Mini,Vaishali,Punjabi Bagh,7.1,20.4,640.0,41.24,4.0,4.1,UPI


Podemos ver que ya los primeros 5 tienen status de 'Completed', sin embargo para asegurar, podemos ver los valores únicos que tiene.

In [26]:
df['Booking Status'].unique()

array(['Completed'], dtype=object)

In [27]:
df['Booking Status'].value_counts()

,count
Booking Status,
Completed,93000


Ya podemos ver que tenemos un dataframe centrado en viajes completados.

#### Limpieza de columnas



---



Sigue eliminar las columnas que no tengan relación con los viajes completados. Estas son las que por descarte podemos saber, ya que se tratan sobre viajes cancelados o incompletos:
*   Cancelled Rides by Customer
*   Reason for cancelling by Customer
*   Cancelled Rides by Driver
*   Driver Cancellation Reason
*   Incomplete Rides
*   Incomplete Rides Reason







In [17]:
columnas_a_eliminar = [
    'Cancelled Rides by Customer',
    'Reason for cancelling by Customer',
    'Cancelled Rides by Driver',
    'Driver Cancellation Reason',
    'Incomplete Rides',
    'Incomplete Rides Reason'
]

df.drop(columns=columnas_a_eliminar, inplace=True)
print(f"Columnas eliminadas")

Columnas eliminadas


Volvemos a mostrar en pequeño resumen del dataframe.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93000 entries, 2 to 149999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             93000 non-null  object 
 1   Time             93000 non-null  object 
 2   Booking ID       93000 non-null  object 
 3   Booking Status   93000 non-null  object 
 4   Customer ID      93000 non-null  object 
 5   Vehicle Type     93000 non-null  object 
 6   Pickup Location  93000 non-null  object 
 7   Drop Location    93000 non-null  object 
 8   Avg VTAT         93000 non-null  float64
 9   Avg CTAT         93000 non-null  float64
 10  Booking Value    93000 non-null  float64
 11  Ride Distance    93000 non-null  float64
 12  Driver Ratings   93000 non-null  float64
 13  Customer Rating  93000 non-null  float64
 14  Payment Method   93000 non-null  object 
dtypes: float64(6), object(9)
memory usage: 11.4+ MB


Ya podemos notar que nos quedamos únicamente con registros que no tengan valores nulos.

In [28]:
df.head()

,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,737.0,48.21,4.1,4.3,UPI
5,2024-02-06,09:44:56,"""CNR4096693""",Completed,"""CID4670564""",Auto,AIIMS,Narsinghpur,5.1,18.1,316.0,4.85,4.1,4.6,UPI
6,2024-06-17,15:45:58,"""CNR2002539""",Completed,"""CID6800553""",Go Mini,Vaishali,Punjabi Bagh,7.1,20.4,640.0,41.24,4.0,4.1,UPI


#### Conversión de fechas a datetime



---



Otra cosa a notar es que tanto date como time son del tipo object, lo cual no es conveniente al trabajar con fechas y tiempos, por lo que se convertirá a tipo datetime.

In [30]:
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors='coerce')
df.drop(['Date', 'Time'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93000 entries, 2 to 149999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Booking ID       93000 non-null  object        
 1   Booking Status   93000 non-null  object        
 2   Customer ID      93000 non-null  object        
 3   Vehicle Type     93000 non-null  object        
 4   Pickup Location  93000 non-null  object        
 5   Drop Location    93000 non-null  object        
 6   Avg VTAT         93000 non-null  float64       
 7   Avg CTAT         93000 non-null  float64       
 8   Booking Value    93000 non-null  float64       
 9   Ride Distance    93000 non-null  float64       
 10  Driver Ratings   93000 non-null  float64       
 11  Customer Rating  93000 non-null  float64       
 12  Payment Method   93000 non-null  object        
 13  Datetime         93000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(6), object(7

Terminamos la limpieza de datos con esto, ya que ahora si logramos ver al final tenemos datetime como datetime.

### Descarga del CSV limpio



---



In [31]:
df.to_csv('datos_limpios.csv', index=False)